In [1]:
file = open("filtered-links_1.txt", "r")
hrefs = []
while True:
    content = file.readline()
    if content:
        hrefs.append(content.strip())  # Using strip() to remove the newline character
    else:
        break

file.close()

file = open("filtered-links_2.txt", "r")
while True:
    content = file.readline()
    if content:
        hrefs.append(content.strip())  # Using strip() again here
    else:
        break

file.close()


In [2]:
print(hrefs[:1]) #there should be 120 links

['https://jlptsensei.com/learn-japanese-vocabulary/%e6%b5%b4%e3%81%b3%e3%82%8b-%e3%81%82%e3%81%b3%e3%82%8b-abiru-meaning/']


In [25]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm

service = webdriver.ChromeService()
driver = webdriver.Chrome(service=service)

data_array = []

for link in tqdm(hrefs[116:]):
    # print("getting link")
    # t = time.time()
    driver.set_page_load_timeout(8)
    try:
        driver.get(link)
    except TimeoutException:
        driver.execute_script("window.stop();")
    # print('Time consuming:', time.time() - t)
    
    # print("extract info")
    # 60% work tbh..need to figure out loading and bot detection but easier to hardcode
        
    vocab_title =   driver.find_element(By.CLASS_NAME, 'd-block.p-3.vocab-title-2.jp').text
    eng_definition =  driver.find_element(By.CLASS_NAME, 'eng-definition.p-lg.mb-lg-5').text
    # Locate the <div> tag by its class name
    
    try:
        div_element = driver.find_element(By.CSS_SELECTOR, "div.goi-notes.p-3.pb-0.bg-light-grey.mb-3.clearfix")
        verb_type = "error"
        # print(div_element.get_attribute('outerHTML'))
        # Now, find all <p> tags within this <div>
        p_tags = div_element.find_elements(By.TAG_NAME, "p")
        # Ensure there are at least three <p> tags
        # print(p_tags)
        # Select the third <p> tag (index 2 since it's 0-based)
        third_p_tag = p_tags[-2]
        
        full_html = third_p_tag.get_attribute('outerHTML')
        # print(full_html)
        # This is a workaround to get the text immediately after the <strong> tag.
        parts = full_html.split('</strong>')
        # print(parts)
        text_after_strong = parts[1]

        parts = text_after_strong.split(', ')

        desired_parts = parts[1:]  # This will give you ["string2", "string3</p>"]
        joined_string = ', '.join(desired_parts)
        verb_type = joined_string
        cleaned_string = joined_string.replace('</p>', '')
        verb_type = cleaned_string
    
        data_array.append((vocab_title, eng_definition, verb_type))
    except:
        data_array.append((vocab_title, eng_definition, "UNKNOWN"))


# Don't forget to close the browser
driver.quit()

        


100%|██████████| 4/4 [00:11<00:00,  2.82s/it]
